In [ ]:
# Setup the environment
!pip install -q -U immutabledict sentencepiece 
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

import sys 
sys.path.append("/kaggle/working/gemma_pytorch/") 
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch

# Load the model
VARIANT = "2b-v2"
MACHINE_TYPE = "cuda" 
model_path = '/kaggle/input/gemma-2-2b-jpn-it/pytorch/gemma-2-2b-jpn-it/1/' 
weights_file = os.path.join(model_path, "model.ckpt")

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

model_config = get_model_config(VARIANT)
model_config.tokenizer = os.path.join(model_path, "tokenizer.model")

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  model.load_weights(weights_file)
  model = model.to(device).eval()

# Use the model
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"

prompt = (
  USER_CHAT_TEMPLATE.format(
    prompt="hello"
  )
  + "<start_of_turn>model\n"
)
print(prompt)

result = model.generate(
  prompt,
  device=device,
  output_len=2560,
)
print(result)
